In [ ]:
# Importing desired libraries ==========================================================================================

# pip install pmdarima
from pmdarima import auto_arima
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import log
from math import sqrt
from sklearn.metrics import mean_squared_error

# Importing the data if it is not already imported =====================================================================

try:
    data


except NameError:
    
    data = pd.read_excel('abc.xlsx')
    
else:
    pass

# Creating an empty dataset to deal with time inconsistency =============================================================

try:
    new_data
    
except NameError:
    
    listw = []
    yr = 2017

    while yr < 2021:

        if yr != 2020:

            for i in range(1,53):

                    num = int(str(yr) + '%02d' %i)
                    listw.append(num)
            yr +=1

        else:
            for i in range(1,6):

                    num = int(str(yr) + '%02d' %i)
                    listw.append(num)
            yr +=1        

    dates = []
    startdate = "2017/01/07"
    enddate = ''
    while enddate != '2020-02-01':

        enddate = str(pd.to_datetime(startdate) + pd.DateOffset(days=7)).split(' ')[0]
        dates.append(startdate.replace('/','-'))

        startdate = enddate.replace('-','/')

    dates.append(enddate)

    new_data = pd.DataFrame(listw, dates).reset_index()
    new_data.columns = ['WeekEnding','WeekId']

else:
    pass


# Predicting for partcular ASIN =======================================================================================

sel_asin = input('Enter the name of the ASIN to get the predictions: ')
print ('ASIN entered = ',sel_asin)

confirm = input('Is ASIN Correct? Y or N : ')

while confirm.lower() == 'n':
    
    sel_asin = input('Enter the name of the ASIN to get the predictions: ')
    print ('ASIN entered = ',sel_asin)
    confirm = input('Is ASIN Correct? Y or N : ')


# Taking out subset of Data ===========================================================================================

data_sku = data[data['RetailerSku'] == sel_asin][['WeekId','UnitsSold']].sort_values('WeekId').reset_index(drop = True)

# Joining with dataframe created in previous step to dal with time inconsistency

asin_dataset = pd.merge(new_data, data_sku, how = 'left', on = 'WeekId').fillna(0)


# ============================================= For ASINS Not Being Sold =============================================

if 0.0 in dict(asin_dataset['UnitsSold'].tail(3).value_counts()).keys() :
    
    if dict(asin_dataset['UnitsSold'].tail(3).value_counts())[0.0] == 3:
    
        print('The ASIN is not being sold')
        
        data3 = asin_dataset[['WeekEnding','UnitsSold']].set_index('WeekEnding').astype(float)
        
        
        fig = plt.figure(figsize=(15,8))
        fig.autofmt_xdate()
        plt.xticks(rotation=90)
        ax = fig.add_subplot(111)
        
        plt.plot(data3['UnitsSold'])
        plt.show()
    
    
# ======================================== ASINS having less than 10 DataPoints =========================================

elif len(asin_dataset[asin_dataset['UnitsSold'] != 0].reset_index(drop = True)) < 10:
    
    print ('mean will be the best predicion')
    
    fig = plt.figure(figsize=(15,8))
    fig.autofmt_xdate()
    plt.xticks(rotation=90)
    ax = fig.add_subplot(111)


    plt.plot(data3['UnitsSold'])
    plt.show()


# =============================For ASINS having more than 10 datapoints and are currently being sold======================    


else:
    
    print(len(asin_dataset[asin_dataset['UnitsSold'] != 0].reset_index(drop = True)))
    data2 = asin_dataset[['WeekEnding','UnitsSold']].set_index('WeekEnding').astype(float)
    
    
    for i in range(len(data2['UnitsSold'])):

        if data2['UnitsSold'][i] == 0:
            data2['UnitsSold'][i] = 0

        else:
            data2['UnitsSold'][i]  = log((data2['UnitsSold'][i]),10)
            
    train = data2[:int(0.75*(len(data2)))]
    valid = data2[int(0.75*(len(data2))):]
    
    
    
    stepwise_model = auto_arima(train, start_p=0, start_q=0,
                           max_p=10, max_q=10, m=12,
                           start_P=0, seasonal=True,
                           max_d=10, D=1, max_D = 10, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    print(stepwise_model.aic())
    
    stepwise_model.fit(train)
    
    forecast = stepwise_model.predict(n_periods=len(valid))
    forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])
    
    
    for i in range(len(forecast['Prediction'])):
    
        if forecast['Prediction'][i] == 0:
            forecast['Prediction'][i] = 0

        else:
            forecast['Prediction'][i]  = 10 ** (forecast['Prediction'][i])


    for i in range(len(train['UnitsSold'])):

        if train['UnitsSold'][i] == 0:
            train['UnitsSold'][i] = 0

        else:
            train['UnitsSold'][i]  = 10 ** (train['UnitsSold'][i])

    for i in range(len(valid['UnitsSold'])):

        if valid['UnitsSold'][i] == 0:
            valid['UnitsSold'][i] = 0

        else:
            valid['UnitsSold'][i]  = 10 ** (valid['UnitsSold'][i])
            
            
# Plottig The Results ==================================================================================================            
     

    fig = plt.figure(figsize=(15,8))
    plt.rcParams.update({'font.size': 16})
    ax = fig.add_subplot(111)

    plt.title('Prediction of Units Sold Using ARIMA')
    plt.xlabel('WEEKS')
    plt.ylabel('Units Sold')
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(5)

    plt.plot(train, label='Train', color = 'grey', linewidth=4)
    plt.plot(valid, label='Valid', color = 'red', linewidth=4)
    plt.plot(forecast, label='Prediction', linewidth=4, color = 'dodgerblue')
    plt.legend()
    plt.show() 


#  Evaluating the Timeseries ============================================================================================
    
    rms = sqrt(mean_squared_error(valid,forecast))
    print(rms)
